# Function‑Calling Demo
Transform user intent into structured arguments and execute a Python function.

In [ ]:
!pip -q install openai ipywidgets

OpenAI's function calling feature allows you to define functions that the model can invoke to return structured results.
This example demonstrates how to provide a function schema and let ChatGPT automatically call it.

In [ ]:
import os, openai, json, datetime, ipywidgets as w
from IPython.display import display, Markdown
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'sk-')

def schedule_meeting(date:str, time:str, topic:str):
    dt = datetime.datetime.strptime(f'{date} {time}', '%Y-%m-%d %H:%M')
    return f"📅 '{topic}' on {dt.strftime('%A %b %d, %Y at %I:%M %p')}" 

FUNC_SCHEMA=[{
 'name':'schedule_meeting',
 'description':'Schedule a meeting',
 'parameters':{
   'type':'object',
   'properties':{
       'date':{'type':'string'},
       'time':{'type':'string'},
       'topic':{'type':'string'}},
   'required':['date','time','topic']}
}]

query=w.Text(value='Book a sync about budget next Monday at 14:30.',description='Query:')
run_btn=w.Button(description='Run')
out=w.Output()

def run(_):
    with out:
        out.clear_output()
        res=openai.ChatCompletion.create(model='gpt-4o-mini',
            messages=[{'role':'user','content':query.value}],
            functions=FUNC_SCHEMA,
            temperature=0)
        msg=res.choices[0].message
        if msg.get('function_call'):
            args=json.loads(msg.function_call.arguments)
            display(Markdown('Args:\n```json\n'+json.dumps(args,indent=2)+'\n```'))
            display(Markdown('Result: '+schedule_meeting(**args)))
        else:
            print('No function call.')
run_btn.on_click(run)
display(w.VBox([query, run_btn, out]))